Import

In [10]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\79651\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [1]:
# from web3 import Web3
import pandas as pd
pd.set_option('display.max_rows', 200)
from tqdm import tqdm
import ast #eval() to interpret code
import json
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
tqdm.pandas()
import time
import warnings
warnings.filterwarnings('ignore')
from pandas import Timestamp
# from scipy.optimize import fsolve
import sympy as sp
import scipy.optimize as optimize

In [7]:
df = pd.read_csv('data_all_ilk_all_usr.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

new_col = None
df.insert(loc = 8, column = 'debt', value = new_col)

calculate debt

In [10]:
def calculate_debt(df):
    ''' 
        To find cumulative debts in each transactions by considers these cases

        case I: liquidate case with the whole amount, debt should be 0
        case II: payback with the whole amount, debt should be 0

    '''
    all_usr = list(df['usr'].unique())

    for usr in tqdm(all_usr):
        tab = df[df['usr'] == usr]

        debt = tab['dai'].iat[0]
        for idx, row in tab.iterrows():
            
            if row['user_action'] == 'liquidate' or row['user_action'] == 'payback dai' or row['user_action'] == 'transfer out':
                amount = -row['dai']

                if amount > debt:
                    debt = 0.0
                    df['debt'].loc[idx] = debt
                else:
                    debt += row['dai']
                    df['debt'].loc[idx] = debt
                    
            else:
                debt += row['dai']
                df['debt'].loc[idx] = debt
                

    return df

In [14]:
programs = ['ETH-A', 'GUNIV3DAIUSDC2-A', 'WBTC-A', 'ETH-C', 'ETH-B']
for prog in programs:
    if prog == 'ETH-A':
        debt = calculate_debt(df[df['ilk'] == prog])
    else:
        df_new = calculate_debt(df[df['ilk'] == prog])
        debt = pd.concat([debt, df_new], ignore_index=True, sort=False)

100%|██████████| 1094/1094 [00:54<00:00, 19.96it/s]


In [15]:
debt.to_csv('debt_all_programs_all_usr.csv')

In [16]:
debt = pd.read_csv('debt_all_programs_all_usr.csv')

Calulate LER

In [18]:
def define_status(debt_amount, usr_act):
    '''  
        To define 4 status of debts:
            liquidate - liquidate by the whole amount of debt(debt = 0)
            restruct - liquidate by the partial amount of debt(debt > 0)
            repaid - user payback debt in the whole amount(debt = 0)
            transfer out - user transfer debt out in the whole amount(debt = 0)
            active - debt is still active(debt > 0)
    '''

    if usr_act == 'liquidate':
        if debt_amount <= 0:
            status = 'liquidate'
        else:
            status = 'restruct'

    elif usr_act == 'payback dai':
        if debt_amount <= 0:
            status = 'repaid'
        else:
            status = 'active'
            
    else:
        if debt_amount > 0:
            status = 'active'
        else:
            status = 'transfer out'

    return status

In [19]:
def define_status_by_row(row):
    '''  
        To define 4 status of debts:
            liquidate - liquidate by the whole amount of debt(debt = 0)
            restruct - liquidate by the partial amount of debt(debt > 0)
            repaid - user payback debt in the whole amount(debt = 0)
            transfer out - user transfer debt out in the whole amount(debt = 0)
            active - debt is still active(debt > 0)
    '''
    usr_act = row['user_action']
    debt_amount = row['debt']
    if usr_act == 'liquidate':
        if debt_amount <= 0:
            status = 'liquidate'
        else:
            status = 'restruct'

    elif usr_act == 'payback dai':
        if debt_amount <= 0:
            status = 'repaid'
        else:
            status = 'active'
            
    else:
        if debt_amount > 0:
            status = 'active'
        else:
            status = 'transfer out'

    return status

In [20]:
new_col = None
debt.insert(loc = 8, column = 'status', value = new_col)
debt['status'] = debt.apply(define_status_by_row, axis=1)

In [21]:
def LER(df, status, th=1000):
    '''   
        Calculate Log Equivalent Rate(LER)
    '''
    if len(df) > th:
        return 0
    else:
        x = sp.symbols('x')

        df = df.reset_index(drop=True)
        acts = list(df['user_action'])

        con_1 = df.shape[0] >= 2
        con_2 = 'generate dai' in acts
        con_3 = 'transfer in' in acts

        if con_1 and (con_2 or con_3):

            if status == 'liquidate' or status == 'repaid':
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                
                #if table end with liquidate/payback dai transaction
                if df['user_action'].iat[-1] == 'liquidate' or df['user_action'].iat[-1] == 'payback dai':
                    end_time = df['timestamp'].iat[-1]
                    end_idx = df['timestamp'].index[-1]

                    debt_ = -(df['dai'].iat[-1] + df['fee'].iat[-1]) 

                    price = df['ETH_price'].iat[-1]
                    pre_coll = df['lock_collateral'].iat[-1]
                    post_coll = 0

                #if table end with return assets
                else:
                    end_time = df['timestamp'].iat[-2]
                    end_idx = df['timestamp'].index[-2]

                    debt_ = -(df['dai'].iat[-2] + df['fee'].iat[-2])

                    price = df['ETH_price'].iat[-2]
                    pre_coll = df['lock_collateral'].iat[-2]
                    post_coll = df['lock_collateral'].iat[-1]
                
                df['time_secs'] = df['timestamp'].apply(lambda y: (end_time - y).total_seconds())
                # df['test'] = df['time_secs'].apply(lambda z: sp.exp(x*z))
                # df['test2'] = df['test'] * df['dai']
                # val = df['test2'].values
                # result = val[val != 0][:-1].sum()
                        
                result = 0
                for idx, row in df.iterrows(): 
                    if idx == end_idx:
                        break
                    result += row['dai'] * sp.exp(x * row['time_secs']) 

                # # result = np.sum(df['dai'].values * sp.exp(x * df['time_secs'].values))
                # # result = sum([z * sp.exp(x * y) for (y, z) in zip(df['time_secs'].values, df['dai'].values)])
                # # result = sum(result)

                if status == 'liquidate':
                    a_t_minus = -pre_coll * price
                    a_t =  post_coll * price

                    loss = -(a_t - a_t_minus)
                    result = result - (debt_ + loss)

                else: 
                    # display(df)
                    # print(status)
                    result = result - debt_
                
                x_init = 0
                result_func = sp.lambdify(x, result, 'numpy')
                opt_x = optimize.fsolve(result_func, x_init)
                

                # print(f'optx: {opt_x}')
                
                if opt_x < 0:
                    rate = 0
                
                else:
                    period_time = (df['time_secs'].iat[0] / 86400)
                    rate = np.exp(opt_x *60 *60 *24 *period_time )*100 - 100
                    rate = rate[0]

                    # print(f'rate: {rate}')

                return rate

            else:
                return None
                
        else:
            return None


In [22]:
def split_tab(df):  
    '''  
        Each of user table can contain more than one debt 
        This function is to split each debt within a table

        return:
            data frame with period of debts and status of debts and interest rate for each debts
    '''
    
    all_usrs = list(df['usr'].unique())

    cols = ['usr', 'start_date', 'end_date', 'status', 'LER', 'debt_number']
    results = pd.DataFrame(columns=cols)
    count = 0
    for usr in tqdm(all_usrs):
        usr_tab = df[df['usr'] == usr]
        acts = list(usr_tab['user_action'])

        usr_tab = usr_tab.reset_index(drop=True)  

        # case 1: if no liquidate or payback with 0 debt in table
        num_idxs = np.where((usr_tab['user_action'] == 'liquidate') | 
                        ((usr_tab['user_action'] == 'payback dai') & (usr_tab['debt'] <= 0))
                        )[0]
        
        if len(num_idxs) == 0:
            debt_tab = usr_tab
            debt = debt_tab['debt'].iat[-1]
            status = define_status(debt, 'in progress')

            ler = LER(debt_tab, status)
            start_date = debt_tab['timestamp'].iat[0]
            end_date = debt_tab['timestamp'].iat[-1]

            data_tab = pd.DataFrame(data = [[usr, start_date, end_date, status, ler, count]], columns=cols)
            results = pd.concat([results, data_tab], ignore_index=True)
            count += 1
            continue

        # case 2: liquidate or payback within table
        start_idx = 0
        for idx in range(0, len(acts)):

            if acts[idx] == 'liquidate':
                # After liquidate process there'll be asset return from auction
                if idx != len(acts)-1 and acts[idx+1] == 'lock asset':
                    debt_tab = usr_tab.loc[start_idx: idx+1]
                    start_idx = idx+2
                    debt = debt_tab['debt'].iat[-2]
                    status = define_status(debt, 'liquidate')                 
                    ler = LER(debt_tab, status)

                    start_date = debt_tab['timestamp'].iat[0]
                    end_date = debt_tab['timestamp'].iat[-2]

                    data_tab = pd.DataFrame(data = [[usr, start_date, end_date, status, ler, count]], columns=cols)
                    results = pd.concat([results, data_tab], ignore_index=True)
                    count += 1

                # In case that there's one return asset 
                else:
                    debt_tab = usr_tab.loc[start_idx: idx]
                    start_idx = idx+1
                    debt = debt_tab['debt'].iat[-1]
                    status = define_status(debt, 'liquidate')  
                    ler = LER(debt_tab, status)

                    start_date = debt_tab['timestamp'].iat[0]
                    end_date = debt_tab['timestamp'].iat[-1]

                    data_tab = pd.DataFrame(data = [[usr, start_date, end_date, status, ler, count]], columns=cols)
                    results = pd.concat([results, data_tab], ignore_index=True)
                    count += 1   

            elif acts[idx] == 'payback dai' and usr_tab['debt'].loc[idx] <= 0:
                debt_tab = usr_tab.loc[start_idx: idx]
                start_idx = idx+1

                debt = debt_tab['debt'].iat[-1]
                status = define_status(debt, 'payback dai')  

                ler = LER(debt_tab, status)
                start_date = debt_tab['timestamp'].iat[0]
                end_date = debt_tab['timestamp'].iat[-1]

                data_tab = pd.DataFrame(data = [[usr, start_date, end_date, status, ler, count]], columns=cols)
                results = pd.concat([results, data_tab], ignore_index=True)
                count += 1
             
         # case 3: there're transaction after the last liquidate or payback
        if num_idxs[-1] != usr_tab.index[-1]:
            debt_tab = usr_tab.loc[start_idx:usr_tab.index[-1]]

            con_1 = debt_tab['user_action'] == 'generate dai'
            con_2 = debt_tab['user_action'] == 'transfer in'

            temp_tab = debt_tab[con_1|con_2]
            if temp_tab.shape[0] >= 1:
                debt_tab = debt_tab

                debt = debt_tab['debt'].iat[-1]
                status = define_status(debt, 'in progress')               
                ler = LER(debt_tab, status)

                start_date = debt_tab['timestamp'].iat[0]
                end_date = debt_tab['timestamp'].iat[-1]

                data_tab = pd.DataFrame(data = [[usr, start_date, end_date, status, ler, count]], columns=cols)
                results = pd.concat([results, data_tab], ignore_index=True)
                count += 1


    return results
            

split table

In [ ]:
programs = ['ETH-A', 'GUNIV3DAIUSDC2-A', 'WBTC-A', 'ETH-C', 'ETH-B']
for prog in programs:
    if prog == 'ETH-A':
        debt_status = split_tab(debt[debt['ilk'] == prog])
    else:
        df_new = split_tab(debt[debt['ilk'] == prog])
        debt_status = pd.concat([debt_status, df_new], ignore_index=True, sort=False)

In [24]:
debt_status_eth_a = split_tab(debt[debt['ilk'] == 'ETH-A'])
debt_status_gun_a = split_tab(debt[debt['ilk'] == 'GUNIV3DAIUSDC2-A'])
debt_status_wbtc_a = split_tab(debt[debt['ilk'] == 'WBTC-A'])
debt_status_eth_c = split_tab(debt[debt['ilk'] == 'ETH-C'])
debt_status_eth_b = split_tab(debt[debt['ilk'] == 'ETH-B'])

100%|██████████| 1094/1094 [01:46<00:00, 10.27it/s]


In [25]:
debt_status_eth_a.to_csv('debt_status_eth_a.csv')
debt_status_gun_a.to_csv('debt_status_gun_a.csv')
debt_status_wbtc_a.to_csv('debt_status_wbtc_a.csv')
debt_status_eth_c.to_csv('debt_status_eth_c.csv')
debt_status_eth_b.to_csv('debt_status_eth_b.csv')

In [ ]:
debt_status.to_csv('debt_table_all_usr.csv')